# Getting Started with Agentune Simulate - Quick Start

This notebook provides a streamlined introduction to the Agentune Simulate library. You'll learn to:

- Load conversation data and extract outcomes
- Set up RAG-based simulation with in-memory vector store
- Run simulations and analyze results

**Note**: This is a simplified version. For more detailed examples including persistent storage, see `getting_started.ipynb`.

## Setup and Imports

In [1]:
import os
import getpass
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from agentune.simulate.models import Outcomes
from agentune.simulate.rag import conversations_to_langchain_documents
from agentune.simulate.simulation.session_builder import SimulationSessionBuilder
from utils import setup_logging_and_asyncio, load_data_with_outcomes

## API Key Configuration

In [2]:
# Set up OpenAI API key
if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

print("✓ API key configured")

Enter your OpenAI API key:  ········


✓ API key configured


## Environment Setup

In [3]:
# Configure logging and asyncio for Jupyter
setup_logging_and_asyncio()

✓ Logging configured
✓ Asyncio event loop configured for Jupyter


## Load Data and Extract Outcomes

In [4]:
# Load conversations and extract outcomes in one step
conversations, outcomes_tuple = load_data_with_outcomes("data/sample_conversations.csv")
outcomes = Outcomes(outcomes=outcomes_tuple)

Loading conversations from data/sample_conversations.csv...
✓ Loaded 100 conversations
✓ Sample conversation has 6 messages
✓ Extracted 2 unique outcomes
  - resolved: Issue was successfully resolved
  - unresolved: Issue was not resolved


## Create Models and Vector Store

In [5]:
# Setup models
chat_model = ChatOpenAI(model="gpt-4o", temperature=0.7)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Create in-memory vector store
documents = conversations_to_langchain_documents(conversations)
vector_store = InMemoryVectorStore.from_documents(documents, embeddings)

print(f"✓ Created vector store with {len(documents)} documents")

✓ Created vector store with 409 documents


## Run Simulation

In [6]:
# Build and run simulation session
session = SimulationSessionBuilder(
    default_chat_model=chat_model,
    outcomes=outcomes,
    vector_store=vector_store,
    max_messages=10
).build()

# Run simulation with first 5 conversations as starting points
base_conversations = conversations[:5]
result = await session.run_simulation(real_conversations=base_conversations)

print("✓ Simulation completed!")

2025-07-16 16:39:13,509 - Starting intent extraction on 5 conversations
2025-07-16 16:39:17,966 - Finished extracting original intents; generated 5 scenarios
2025-07-16 16:39:17,967 - Starting conversation simulations (self.max_concurrent_conversations=20)
2025-07-16 16:39:22,968 - Progress: 0/5 scenarios completed
2025-07-16 16:39:52,973 - Progress: 2/5 scenarios completed
2025-07-16 16:39:57,975 - Progress: 4/5 scenarios completed
2025-07-16 16:40:07,979 - Progress: 5/5 scenarios completed
2025-07-16 16:40:07,980 - Finished simulating conversations; simulated 5 conversations, with 0 failures
2025-07-16 16:40:07,980 - Starting analysis of simulation results
2025-07-16 16:40:16,483 - Finished analyzing results


✓ Simulation completed!


## Analyze Results

In [7]:
# Display comprehensive results
print(result.generate_summary())

# Save results to file
output_file = "simulation_results.json"
result.save_to_file(output_file)
print(f"\n✓ Results saved to {output_file}")

SIMULATION RESULTS
Session name: Simulation Session
Original conversations: 5
Simulated conversations: 5

Average messages per conversation:
  Original: 4.4
  Simulated: 10.0

Outcome distribution comparison:
Outcome              Original        Simulated      
--------------------------------------------------
resolved               4 (80.0%)     5 (100.0%)
unresolved             1 (20.0%)     0 ( 0.0%)

Sample conversation (10 messages):
  1. customer: Last night, I waited in line for 2 hours in the business office, but because I only had a copy of my...
  2. agent: I'm really sorry for the inconvenience you've experienced. To resolve this, please make sure to brin...
  3. customer: This is absolutely ridiculous! How many more hoops do I need to jump through before I can cancel the...
  4. agent: I'm really sorry for the frustration this has caused. I'll escalate your issue internally to ensure ...
  ... and 6 more messages

✓ Results saved to simulation_results.json


## Next Steps

Now that you've completed a basic simulation:

1. **Scale up**: Try with more conversations and longer interactions
2. **Use your own data**: Convert your data to the CSV format shown in `data/sample_conversations.csv`
3. **Advanced features**: See `getting_started.ipynb` for persistent storage with Chroma and more detailed explanations
4. **Web interface**: Try the Streamlit app at `streamlit/app.py`

### For your own data:
Create a CSV with columns: `conversation_id`, `sender`, `content`, `timestamp`, `outcome_name`, `outcome_description`

### Resources:
- [Full documentation](../README.md)
- [Complete examples](../examples/)
- [Streamlit web interface](../streamlit/)